# Extracting IMF data using API
## Usage of the function

In [1]:
import imf_api

In [2]:
trading_data, missing_country = imf_api.extract_export_data('US')
trading_data

,2019,2020
AT,5747.8,3431.9
BT,2.8,2.9
AO,536.2,470.5
HK,30800.0,23986.5
CL,15776.2,12766.6
...,...,...
1C_080,71448.0,52881.9
A10,417574.7,342118.5
E1,30128.3,26991.8
U2,245221.4,213554.6


In [3]:
imf_api.country_codes

{'AF': 'Afghanistan',
 'F19': 'Africa not allocated',
 'AL': 'Albania',
 'DZ': 'Algeria',
 'AS': 'American Samoa',
 'AO': 'Angola',
 'AI': 'Anguilla',
 'AG': 'Antigua and Barbuda',
 'AR': 'Argentina',
 'AM': 'Armenia',
 'AW': 'Aruba',
 'S19': 'Asia not allocated',
 'AU': 'Australia',
 'AT': 'Austria',
 'AZ': 'Azerbaijan',
 'BS': 'Bahamas',
 'BH': 'Bahrain',
 'BD': 'Bangladesh',
 'BB': 'Barbados',
 'BY': 'Belarus',
 'BE': 'Belgium',
 'R1': 'Belgo-Luxembourg Economic Union',
 'BZ': 'Belize',
 'BJ': 'Benin',
 'BM': 'Bermuda',
 'BT': 'Bhutan',
 'BO': 'Bolivia',
 'BA': 'Bosnia and Herzegovina',
 'BW': 'Botswana',
 'BR': 'Brazil',
 'BN': 'Brunei Darussalam',
 'BG': 'Bulgaria',
 'BF': 'Burkina Faso',
 'BI': 'Burundi',
 'CV': 'Cabo Verde',
 'KH': 'Cambodia',
 'CM': 'Cameroon',
 'CA': 'Canada',
 'CF': 'Central African Republic',
 'TD': 'Chad',
 'CL': 'Chile',
 'HK': 'Hong Kong, China',
 'MO': 'Macao',
 'CN': 'China',
 'CO': 'Colombia',
 'KM': 'Comoros',
 'CD': 'Congo, the Democratic Republic of

# Brief explanation about imf_api.py.  
Reference: https://www.bd-econ.com/imfapi1.html
## Find the dimensions

Key should be
- CompactData/{database ID}/{frequency}
- .{item1 from dimension1}
- .{item1 from dimension2}
- .{item1 from dimension3}
- .{item1 from dimension4}
- ?startPeriod={start date}&endPeriod={end date}
First, find the dimensions and contants.

In [4]:
import requests
import pandas as pd

In [5]:
url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
key = 'DataStructure/DOT'
dimension_list = requests.get(f'{url}{key}').json()\
            ['Structure']['KeyFamilies']['KeyFamily']\
            ['Components']['Dimension']
for n, dimension in enumerate(dimension_list):
    print(f'Dimension {n+1}: {dimension["@codelist"]}')

Dimension 1: CL_FREQ
Dimension 2: CL_AREA_DOT
Dimension 3: CL_INDICATOR_DOT
Dimension 4: CL_COUNTERPART_AREA_DOT


## Find the codes for each dimension
### Dimension 1: CL_FREQ

In [6]:
key = f'CodeList/{dimension_list[0]["@codelist"]}'
code_list_d1 = requests.get(f'{url}{key}').json()\
	    ['Structure']['CodeLists']['CodeList']['Code']
for code in code_list_d1:
    print(f"{code['Description']['#text']}: {code['@value']}")

Annual: A
Bi-annual: B
Quarterly: Q
Monthly: M
Daily: D
Weekly: W


### Dimension 2: CL_AREA_DOT
Storing the country names and its codes to the 'country_codes' dictionary.

In [7]:
key = f'CodeList/{dimension_list[1]["@codelist"]}'
code_list_d2 = requests.get(f'{url}{key}').json()\
	    ['Structure']['CodeLists']['CodeList']['Code']
country_codes = {}
for code in code_list_d2:
    country_codes[code['@value']] = code['Description']['#text']
country_codes

{'AF': 'Afghanistan',
 'F19': 'Africa not allocated',
 'AL': 'Albania',
 'DZ': 'Algeria',
 'AS': 'American Samoa',
 'AO': 'Angola',
 'AI': 'Anguilla',
 'AG': 'Antigua and Barbuda',
 'AR': 'Argentina',
 'AM': 'Armenia',
 'AW': 'Aruba',
 'S19': 'Asia not allocated',
 'AU': 'Australia',
 'AT': 'Austria',
 'AZ': 'Azerbaijan',
 'BS': 'Bahamas',
 'BH': 'Bahrain',
 'BD': 'Bangladesh',
 'BB': 'Barbados',
 'BY': 'Belarus',
 'BE': 'Belgium',
 'R1': 'Belgo-Luxembourg Economic Union',
 'BZ': 'Belize',
 'BJ': 'Benin',
 'BM': 'Bermuda',
 'BT': 'Bhutan',
 'BO': 'Bolivia',
 'BA': 'Bosnia and Herzegovina',
 'BW': 'Botswana',
 'BR': 'Brazil',
 'BN': 'Brunei Darussalam',
 'BG': 'Bulgaria',
 'BF': 'Burkina Faso',
 'BI': 'Burundi',
 'CV': 'Cabo Verde',
 'KH': 'Cambodia',
 'CM': 'Cameroon',
 'CA': 'Canada',
 'CF': 'Central African Republic',
 'TD': 'Chad',
 'CL': 'Chile',
 'HK': 'Hong Kong, China',
 'MO': 'Macao',
 'CN': 'China',
 'CO': 'Colombia',
 'KM': 'Comoros',
 'CD': 'Congo, the Democratic Republic of

### Dimension 3: CL_INDICATOR_DOT

In [8]:
key = f"CodeList/{dimension_list[2]['@codelist']}"
code_list_d3 = requests.get(f'{url}{key}').json()\
	    ['Structure']['CodeLists']['CodeList']['Code']
for code in code_list_d3:
    print(f"{code['Description']['#text']}: {code['@value']}")

Goods, Value of Exports, Free on board (FOB), US Dollars: TXG_FOB_USD
Goods, Value of Imports, Cost, Insurance, Freight (CIF), US Dollars: TMG_CIF_USD
Goods, Value of Imports, Free on board (FOB), US Dollars: TMG_FOB_USD
Goods, Value of Trade Balance, US Dollars: TBG_USD


### Dimension 4: CL_COUNTERPART_AREA_DOT
This is same to the dimension 2.

In [9]:
key = f"CodeList/{dimension_list[3]['@codelist']}"
code_list4 = requests.get(f'{url}{key}').json()\
	    ['Structure']['CodeLists']['CodeList']['Code']
for code in code_list4:
    print(f"{code['Description']['#text']}: {code['@value']}")

Afghanistan: AF
Africa not allocated: F19
Albania: AL
Algeria: DZ
American Samoa: AS
Angola: AO
Anguilla: AI
Antigua and Barbuda: AG
Argentina: AR
Armenia: AM
Aruba: AW
Asia not allocated: S19
Australia: AU
Austria: AT
Azerbaijan: AZ
Bahamas: BS
Bahrain: BH
Bangladesh: BD
Barbados: BB
Belarus: BY
Belgium: BE
Belgo-Luxembourg Economic Union: R1
Belize: BZ
Benin: BJ
Bermuda: BM
Bhutan: BT
Bolivia: BO
Bosnia and Herzegovina: BA
Botswana: BW
Brazil: BR
Brunei Darussalam: BN
Bulgaria: BG
Burkina Faso: BF
Burundi: BI
Cabo Verde: CV
Cambodia: KH
Cameroon: CM
Canada: CA
Central African Republic: CF
Chad: TD
Chile: CL
Hong Kong, China: HK
Macao: MO
China: CN
Colombia: CO
Comoros: KM
Congo, the Democratic Republic of the: CD
Congo: CG
Costa Rica: CR
Cote d`Ivoire: CI
Not allocated/unspecified: _X
Croatia: HR
Cuba: CU
Curacao: CW
Cyprus: CY
Czech Republic: CZ
Former Czechoslovakia: CSH
Denmark: DK
Djibouti: DJ
Dominica: DM
Dominican Republic: DO
East Germany: DE2
Ecuador: EC
Egypt: EG
El Salvador

## Sample Codes

In [10]:
key = f'CompactData/DOT/A.JP.TXG_FOB_USD.W00+B0?startPeriod=2019&endPeriod=2020'
data = requests.get(f'{url}{key}').json()['CompactData']['DataSet']['Series']
df_dict = {}
for s in data:
    df_dict_col = {}
    for i in s['Obs']:
        df_dict_col[i['@TIME_PERIOD']] = round(float(i['@OBS_VALUE']), 1)
    df_dict[s['@COUNTERPART_AREA']] = df_dict_col
df = pd.DataFrame(df_dict)
df

,W00,B0
2019,705632.0,68303.0
2020,638167.0,58823.7


### Obtaining the metadata

In [11]:
key = 'GenericMetadata/IFS/M.GB.PMP_IX'
metadata = requests.get(f'{url}{key}').json()
country = metadata['GenericMetadata']['MetadataSet']\
            ['AttributeValueSet'][1]['ReportedAttribute']\
            [1]['ReportedAttribute'][3]['Value']['#text']
indicator = metadata['GenericMetadata']['MetadataSet']\
            ['AttributeValueSet'][2]['ReportedAttribute']\
            [1]['ReportedAttribute'][3]['Value']['#text']

'''
# Checking the contents of the 'metadata' dictionary
pprint.pprint(metadata['GenericMetadata']['MetadataSet']\
            ['AttributeValueSet'][2]['ReportedAttribute']\
            [1]['ReportedAttribute'])
pprint.pprint(metadata['GenericMetadata']['MetadataSet']\
            ['AttributeValueSet'][1]['ReportedAttribute'])
'''

print(f'Country: {country}\nIndicator: {indicator}')

Country: United Kingdom
Indicator: Prices, Import Price Index, All Commodities, Index
